In [1]:
import requests
import cloudscraper # Para manejar posibles protecciones de Cloudflare
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import time
from random import uniform
import os

print("Librerías importadas y listas para usar.")

Librerías importadas y listas para usar.


Función para Obtener Contenido HTML (fetch_html_content)

In [2]:
def fetch_html_content(url):
    """
    Obtiene el contenido HTML de una URL usando cloudscraper.

    Args:
        url (str): La URL de la página a obtener.

    Returns:
        requests.models.Response: El objeto de respuesta de la solicitud,
                                   o None si ocurre un error significativo.
    """
    print(f"Fetching HTML desde: {url}")
    try:
        scraper = cloudscraper.create_scraper(
            browser={
                'browser': 'chrome',
                'platform': 'windows',
                'mobile': False
            }
        )
    except Exception as e:
        print(f"Error al crear la instancia de cloudscraper: {e}. Intentando con requests.get().")
        try:
            time.sleep(uniform(2, 4)) # Pausa antes de reintentar
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=30) # Aumentado timeout
            response.raise_for_status()
            print(f"Contenido obtenido con requests.get() (Código de estado: {response.status_code})")
            return response
        except requests.exceptions.HTTPError as http_err_req:
            print(f"Error HTTP (requests.get) al acceder a {url}: {http_err_req} (Código: {response.status_code if 'response' in locals() and response else 'N/A'})")
            return None
        except requests.exceptions.RequestException as req_err_req:
            print(f"Error de red/conexión (requests.get) al acceder a {url}: {req_err_req}")
            return None

    time.sleep(uniform(2.5, 5.5)) # Aumentar ligeramente la pausa respetuosa

    response = None
    try:
        response = scraper.get(url, timeout=30) # Aumentado timeout
        response.raise_for_status() 
        print(f"Contenido obtenido exitosamente con cloudscraper (Código de estado: {response.status_code})")
    except requests.exceptions.HTTPError as http_err:
        print(f"Error HTTP (cloudscraper) al acceder a {url}: {http_err} (Código: {response.status_code if response else 'N/A'})")
        return None
    except requests.exceptions.RequestException as req_err:
        print(f"Error de red/conexión (cloudscraper) al acceder a {url}: {req_err}")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado (cloudscraper) al acceder a {url}: {e}")
        return None
        
    return response

print("Función fetch_html_content definida.")

Función fetch_html_content definida.


Función para Obtener Datos Generales de Partidos de una LIGA (get_understat_league_general_match_data)

In [3]:
def get_understat_league_general_match_data(league_name_url, year, fetch_function):
    """
    Extrae la lista general de partidos de una LIGA para una temporada específica de Understat.com.
    Esta función obtiene la variable 'datesData' de la página de la LIGA.

    Args:
        league_name_url (str): El nombre de la liga tal como se usa en la URL de Understat (ej. "La_liga").
        year (int): El año de inicio de la temporada.
        fetch_function (function): La función a usar para obtener el contenido HTML.

    Returns:
        list: Una lista de diccionarios (datos crudos de 'datesData'), o None si falla.
    """
    url = f"https://understat.com/league/{league_name_url}/{year}" # URL para la página de la liga
    
    response = fetch_function(url)

    if response is None or response.status_code != 200:
        return None

    html_content = response.text
    soup = BeautifulSoup(html_content, 'lxml')
    
    script_tags = soup.find_all('script')
    match_data_string = None

    for script in script_tags:
        if script.string and 'datesData' in script.string: # 'datesData' en la página de la liga contiene todos los partidos
            match = re.search(r"datesData\s*=\s*JSON\.parse\s*\(\s*'(.*?)'\s*\)", script.string)
            if match:
                match_data_string = match.group(1)
                break
    
    if not match_data_string:
        print(f"No se pudo encontrar la cadena de datos 'datesData' en la página de la liga {league_name_url} para el año {year}.")
        return None

    try:
        processed_string = bytes(match_data_string, "utf-8").decode("unicode_escape")
        data = json.loads(processed_string)
    except Exception as e:
        print(f"Error al decodificar o parsear JSON para datesData (liga: {league_name_url}, año: {year}): {e}")
        return None
            
    if data:
        print(f"Datos generales de partidos extraídos para la liga {league_name_url} {year}. {len(data)} partidos encontrados.")
        return data 
    else:
        print(f"No se encontraron datos de partidos en el JSON parseado para la liga {league_name_url} {year}.")
        return None

print("Función get_understat_league_general_match_data definida.")

Función get_understat_league_general_match_data definida.


Función para Obtener Datos Detallados de un Partido Individual

In [4]:
def get_understat_single_match_detailed_data(match_id, fetch_function):
    """
    Extrae datos detallados (incluyendo 'shots_data' y 'rosters_data')
    de una página de partido individual de Understat.com.
    """
    url = f"https://understat.com/match/{match_id}"
    
    response = fetch_function(url)

    if response is None or response.status_code != 200:
        return None

    html_content = response.text
    soup = BeautifulSoup(html_content, 'lxml')
    
    script_tags = soup.find_all('script')
    
    extracted_data = {
        "shots_data": None,
        "rosters_data": None 
    }

    patterns = {
        "shots_data": r"shotsData\s*=\s*JSON\.parse\s*\(\s*'(.*?)'\s*\)",
        "rosters_data": r"rostersData\s*=\s*JSON\.parse\s*\(\s*'(.*?)'\s*\)"
    }

    for script in script_tags:
        if script.string:
            for data_key, pattern in patterns.items():
                if extracted_data[data_key] is not None: 
                    continue
                match = re.search(pattern, script.string)
                if match:
                    data_string = match.group(1)
                    try:
                        processed_string = bytes(data_string, "utf-8").decode("unicode_escape")
                        extracted_data[data_key] = json.loads(processed_string)
                        print(f"  Datos para '{data_key}' (partido {match_id}) parseados exitosamente.")
                    except Exception as e:
                        print(f"  Error al decodificar JSON para {data_key} (partido {match_id}): {e}")
                        
    if extracted_data.get("rosters_data") is None: 
        print(f"  No se pudo extraer 'rosters_data' de la página del partido {match_id}.")
        return None 
        
    return extracted_data

print("Función get_understat_single_match_detailed_data definida.")

Función get_understat_single_match_detailed_data definida.


Función Principal para Orquestar el Scraping por Liga y Temporadas

In [5]:
def scrape_league_data_for_model(league_url_name, league_print_name, start_year, end_year, fetch_function):
    """
    Orquesta la obtención de datos de partidos de una liga para un rango de temporadas
    y los procesa para crear un DataFrame con las columnas deseadas.
    """
    all_matches_for_all_seasons = []

    for year in range(start_year, end_year + 1):
        print(f"\n--- Iniciando scraping para {league_print_name} - Temporada {year}/{year+1} ---")
        
        # 1. Obtener la lista general de partidos para la LIGA y la temporada
        general_match_list_for_league = get_understat_league_general_match_data(
            league_name_url=league_url_name,
            year=year,
            fetch_function=fetch_function
        )

        if not general_match_list_for_league:
            print(f"No se pudo obtener la lista general de partidos para {league_print_name} {year}. Omitiendo esta temporada.")
            continue

        print(f"\nProcesando {len(general_match_list_for_league)} partidos de {league_print_name} {year} para obtener detalles...")
        for i, general_info in enumerate(general_match_list_for_league):
            match_id = general_info.get('id')
            match_datetime = general_info.get('datetime')

            if not match_id or not general_info.get('isResult'):
                print(f"Omitiendo partido general #{i+1} de la liga (ID: {match_id}) por falta de ID o no ser resultado final.")
                continue

            home_team_name = general_info.get('h', {}).get('title')
            away_team_name = general_info.get('a', {}).get('title')
            home_team_goal = int(general_info.get('goals', {}).get('h', 0))
            away_team_goal = int(general_info.get('goals', {}).get('a', 0))

            if home_team_goal > away_team_goal:
                resultado = 'H'
            elif away_team_goal > home_team_goal:
                resultado = 'A'
            else:
                resultado = 'D'
            
            print(f"\nObteniendo detalles para partido ID: {match_id} ({i+1}/{len(general_match_list_for_league)}) - {home_team_name} vs {away_team_name}")
            detailed_payload = get_understat_single_match_detailed_data(match_id, fetch_function)

            home_aggression_proxy = 0
            away_aggression_proxy = 0

            if detailed_payload and detailed_payload.get("rosters_data"):
                rosters = detailed_payload["rosters_data"]
                if 'h' in rosters:
                    for player_id, p_stats in rosters['h'].items():
                        home_aggression_proxy += int(p_stats.get('yellow_card', 0)) * 1
                        home_aggression_proxy += int(p_stats.get('red_card', 0)) * 3
                if 'a' in rosters:
                    for player_id, p_stats in rosters['a'].items():
                        away_aggression_proxy += int(p_stats.get('yellow_card', 0)) * 1
                        away_aggression_proxy += int(p_stats.get('red_card', 0)) * 3
                # print(f"  Agresión Proxy -> Local: {home_aggression_proxy}, Visitante: {away_aggression_proxy}")
            else:
                print(f"  No se pudo obtener rosters_data para el partido {match_id}. Agresión no calculada.")
                home_aggression_proxy = None 
                away_aggression_proxy = None

            all_matches_for_all_seasons.append({
                "season_year_start": year, # Añadimos el año de la temporada
                "league_name": league_print_name, # Añadimos el nombre de la liga
                "match_date": pd.to_datetime(match_datetime).date() if match_datetime else None,
                "home_team_name": home_team_name,
                "away_team_name": away_team_name,
                "home_team_goal": home_team_goal,
                "away_team_goal": away_team_goal,
                "home_aggression": home_aggression_proxy,
                "away_aggression": away_aggression_proxy,
                "resultado": resultado
            })
        print(f"--- Finalizado scraping para {league_print_name} - Temporada {year}/{year+1} ---")


    if not all_matches_for_all_seasons:
        print(f"No se procesaron datos de partidos para la liga {league_print_name} en el rango de años especificado.")
        return pd.DataFrame()
        
    final_df = pd.DataFrame(all_matches_for_all_seasons)
    return final_df

print("Función scrape_league_data_for_model definida.")

Función scrape_league_data_for_model definida.


Ejecución del Proceso para La Liga (2016-2024) y Guardado del CSV

In [ ]:
# --- Configuración del Scraping para La Liga ---
target_league_url_name_param = "La_liga" # Nombre de La Liga para la URL de Understat
target_league_print_name_param = "La Liga" # Nombre para mostrar y para la columna en el DataFrame
# Rango de años de inicio de temporada
# 2016 para 2016/17, ..., 2023 para 2023/24.
# Si quieres la temporada 2024/25 (que inicia en 2024), incluye 2024.
# El año actual es 2025, la temporada 2024/25 ya debería estar avanzada o completa.
start_season_year_param = 2016
end_season_year_param = 2024 # Incluye la temporada que comienza en 2024 (2024/2025)

print(f"--- Iniciando Proceso Completo para {target_league_print_name_param} - Temporadas {start_season_year_param}-{end_season_year_param} ---")

# Llamar a la función principal que orquesta todo
df_la_liga_model = scrape_league_data_for_model(
    league_url_name=target_league_url_name_param,
    league_print_name=target_league_print_name_param,
    start_year=start_season_year_param,
    end_year=end_season_year_param,
    fetch_function=fetch_html_content
)

# Mostrar información del DataFrame resultante y guardarlo
if not df_la_liga_model.empty:
    print("\n\n--- DataFrame Final Generado para La Liga ---")
    print(f"Total de partidos procesados (filas): {df_la_liga_model.shape[0]}")
    print(f"Columnas generadas: {df_la_liga_model.shape[1]}")
    
    print("\nPrimeras 5 filas del DataFrame:")
    pd.set_option('display.max_columns', None)
    print(df_la_liga_model.head())
    
    print("\nÚltimas 5 filas del DataFrame (para ver los años más recientes):")
    print(df_la_liga_model.tail())
    
    print("\nInformación del DataFrame:")
    df_la_liga_model.info()
    
    # Guardar en CSV
    # !!! IMPORTANTE: VERIFICA Y CAMBIA ESTA RUTA SI ES NECESARIO !!!
    output_csv_path_la_liga = r"C:\Users\juand\Downloads\Proyecto_analisis\understat_la_liga_2016-2024_model_data.csv"
    try:
        output_csv_directory = os.path.dirname(output_csv_path_la_liga)
        if output_csv_directory and not os.path.exists(output_csv_directory): # Solo crear si el directorio no es la raíz y no existe
            os.makedirs(output_csv_directory)
            print(f"\nDirectorio para CSV creado: {output_csv_directory}")

        df_la_liga_model.to_csv(output_csv_path_la_liga, index=False, encoding='utf-8-sig')
        print(f"\nDataFrame de La Liga guardado exitosamente en: {output_csv_path_la_liga}")
    except Exception as e:
        print(f"\nError al guardar el CSV de La Liga: {e}")
            
else:
    print("\nEl DataFrame final para La Liga está vacío. No se procesaron datos o hubo errores.")

print(f"\n--- Proceso de Scraping para {target_league_print_name_param} Finalizado ---")

--- Iniciando Proceso Completo para La Liga - Temporadas 2016-2024 ---

--- Iniciando scraping para La Liga - Temporada 2016/2017 ---
Fetching HTML desde: https://understat.com/league/La_liga/2016
Contenido obtenido exitosamente con cloudscraper (Código de estado: 200)
Datos generales de partidos extraídos para la liga La_liga 2016. 380 partidos encontrados.

Procesando 380 partidos de La Liga 2016 para obtener detalles...

Obteniendo detalles para partido ID: 1779 (1/380) - Malaga vs Osasuna
Fetching HTML desde: https://understat.com/match/1779
Contenido obtenido exitosamente con cloudscraper (Código de estado: 200)
  Datos para 'shots_data' (partido 1779) parseados exitosamente.
  Datos para 'rosters_data' (partido 1779) parseados exitosamente.

Obteniendo detalles para partido ID: 1780 (2/380) - Deportivo La Coruna vs Eibar
Fetching HTML desde: https://understat.com/match/1780
Contenido obtenido exitosamente con cloudscraper (Código de estado: 200)
  Datos para 'shots_data' (partido